In [7]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from skimage.util import view_as_windows
from skimage.measure import block_reduce
from tensorflow.keras.datasets import mnist

# 从 cwSaab.py 导入 cwSaab 类
from cwSaab import cwSaab

def pad_images(X, pad_width, pad_height):
    return np.pad(X, pad_width=((0, 0), (pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant', constant_values=0)

# 定义一个函数来获取数据的窗口视图
def Shrink(X, shrinkArg):
    win = shrinkArg['win']
    stride = shrinkArg['stride']
    ch = X.shape[-1]
    X = view_as_windows(X, (1, win, win, ch), (1, stride, stride, ch))
    return X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)

# 定义最大池化函数
def max_pooling(X):
    return block_reduce(X, block_size=(1, 2, 2, 1), func=np.max)
(X, y), (X_test, y_test) = mnist.load_data()
X = X.reshape((-1, 28, 28, 1))


# 分割数据为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 对图像进行填充
pad_size = 2
X_train_padded = pad_images(X_train, pad_size, pad_size)
X_test_padded = pad_images(X_test, pad_size, pad_size)

# 对特征图进行窗口提取和最大池化
X_train_pooled = max_pooling(X_train_padded)
X_test_pooled = max_pooling(X_test_padded)

# 设置 Saab 参数
SaabArgs = [{'num_AC_kernels':-1, 'needBias':True, 'cw':False}, 
            {'num_AC_kernels':5, 'needBias':True, 'cw':True}]
shrinkArgs = [{'func': Shrink, 'win':5, 'stride': 1}, 
              {'func': Shrink, 'win':5, 'stride': 1}]

# 初始化 cwSaab 类
cwsaab = cwSaab(depth=2, TH1=0.005, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs)

# 训练模型
cwsaab.fit(X_train_pooled)

# 变换训练数据和测试数据
train_features = cwsaab.transform(X_train_pooled)
test_features = cwsaab.transform(X_test_pooled)
def unify_feature_size(features):
    # 假设我们以最小的尺寸为基准
    min_shape = min((feature.shape[1], feature.shape[2]) for feature in features)
    new_features = []
    for feature in features:
        if feature.shape[1] > min_shape[0] or feature.shape[2] > min_shape[1]:
            # 简单的裁剪
            feature = feature[:, :min_shape[0], :min_shape[1], :]
        new_features.append(feature)
    return new_features

# 在训练和测试特征上应用尺寸统一
train_features = unify_feature_size(train_features)
test_features = unify_feature_size(test_features)

# 安全地合并特征
train_features_concat = np.concatenate(train_features, axis=-1).reshape(len(X_train_pooled), -1)
test_features_concat = np.concatenate(test_features, axis=-1).reshape(len(X_test_pooled), -1)

# 使用 XGBoost 进行分类
from xgboost import XGBClassifier
classifier = XGBClassifier(n_estimators=100)
classifier.fit(np.concatenate(train_features, axis=-1).reshape(len(X_train_pooled), -1), y_train)

# 测试准确率
test_accuracy = classifier.score(np.concatenate(test_features, axis=-1).reshape(len(X_test_pooled), -1), y_test)
print(f"Test Accuracy: {test_accuracy:.2%}")


=============================================>c/w Saab Train Hop 1
=============================================>c/w Saab Train Hop 2
Test Accuracy: 97.80%


In [9]:
import numpy as np
import time
from sklearn import datasets
from sklearn.model_selection import train_test_split
from skimage.util import view_as_windows
from skimage.measure import block_reduce
from tensorflow.keras.datasets import mnist
from xgboost import XGBClassifier

# 从 cwSaab.py 导入 cwSaab 类
from cwSaab import cwSaab

def pad_images(X, pad_width, pad_height):
    return np.pad(X, pad_width=((0, 0), (pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant', constant_values=0)

def Shrink(X, shrinkArg):
    win = shrinkArg['win']
    stride = shrinkArg['stride']
    ch = X.shape[-1]
    X = view_as_windows(X, (1, win, win, ch), (1, stride, stride, ch))
    return X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)

def max_pooling(X):
    return block_reduce(X, block_size=(1, 2, 2, 1), func=np.max)

(X, y), (X_test, y_test) = mnist.load_data()
X = X.reshape((-1, 28, 28, 1))

# 对图像进行填充
pad_size = 2
X_padded = pad_images(X, pad_size, pad_size)

# 对特征图进行窗口提取和最大池化
X_pooled = max_pooling(X_padded)

# 设置 Saab 参数
SaabArgs = [{'num_AC_kernels':-1, 'needBias':True, 'cw':False}, 
            {'num_AC_kernels':5, 'needBias':True, 'cw':True}]
shrinkArgs = [{'func': Shrink, 'win':5, 'stride': 1}, 
              {'func': Shrink, 'win':5, 'stride': 1}]

# 初始化 cwSaab 类
cwsaab = cwSaab(depth=2, TH1=0.005, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs)

# 训练模型
start_time = time.time()
cwsaab.fit(X_pooled)
end_time = time.time()
training_time = end_time - start_time

# 变换训练数据和测试数据
features = cwsaab.transform(X_pooled)

# 应用尺寸统一
features = unify_feature_size(features)

# 安全地合并特征
features_concat = np.concatenate(features, axis=-1).reshape(len(X_pooled), -1)

# 使用 XGBoost 进行分类
classifier = XGBClassifier(n_estimators=100)
classifier.fit(features_concat, y)

# 计算模型大小
model_size = np.sum([coef.size for coef in classifier.get_booster().get_score(importance_type='weight').values()])

# 打印训练时间和模型大小
print(f"Training time: {training_time:.2f} seconds")
print(f"Model size: {model_size} parameters")


=============================================>c/w Saab Train Hop 1
=============================================>c/w Saab Train Hop 2


AttributeError: 'float' object has no attribute 'size'